# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

Давайте рассмотрим исходные данные внимательнее

In [4]:
data0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Давайте осуществим проверку на наличие дубликатов

In [7]:
data0[data0.duplicated()]

,id,f0,f1,f2,product


In [8]:
data1[data1.duplicated()]

,id,f0,f1,f2,product


In [9]:
data2[data2.duplicated()]

,id,f0,f1,f2,product


Осуществим проверку на наличие дубликатов отдельно по столбцу id

In [10]:
data0[data0.duplicated(subset = ['id'])]

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


In [11]:
data1[data1.duplicated(subset = ['id'])]

,id,f0,f1,f2,product
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [12]:
data2[data2.duplicated(subset = ['id'])]

,id,f0,f1,f2,product
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


Осуществим проверку на наличие пропусков

In [13]:
data0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [14]:
data1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [15]:
data2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Мы можем удалить столбец id, так как он нам не нужен.

In [16]:
data0 = data0.drop('id', axis = 1)
data1 = data1.drop('id', axis = 1)
data2 = data2.drop('id', axis = 1)

Вывод: Мы провели первоначальный анализ данных, включая проверку на пропуски, дубликаты и качество самих данных. В наших выборках не обнаружено полных дубликатов, но есть несколько дубликатов, отличающихся только по идентификатору (id). Решено оставить данные без изменений и продолжить работу с ними.

## Обучение и проверка модели

Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.

Сделаем данные однородными по масштабу.

In [17]:
features0 = data0.drop('product', axis=1)
target0 = data0['product']
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(features0, target0, train_size=0.25, random_state=12345, shuffle=False)


In [18]:
scaler = StandardScaler()
scaler.fit(features_train0)
features_train0 = scaler.transform(features_train0)
features_valid0 = scaler.transform(features_valid0)

In [19]:
features1 = data1.drop('product', axis=1)
target1 = data1['product']
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(features1, target1, train_size=0.25, random_state=12345, shuffle=False)

In [20]:
scaler1 = StandardScaler()
scaler1.fit(features_train1)
features_train1 = scaler.transform(features_train1)
features_valid1 = scaler.transform(features_valid1)

In [21]:
features2 = data2.drop('product', axis=1)
target2 = data2['product']
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(features2, target2, train_size=0.25, random_state=12345, shuffle=False)

In [22]:
scaler2 = StandardScaler()
scaler2.fit(features_train2)
features_train2 = scaler.transform(features_train2)
features_valid2 = scaler.transform(features_valid2)

Далее мы приступим к обучению моделей, получению прогнозов, определению среднего запаса предсказанного сырья и вычислению среднеквадратичной ошибки (RMSE) модели

In [26]:
def rmse_prediction(features_train, target_train, features_valid, target_valid ):
    model = LinearRegression()
    model.fit(features_train,target_train)
    predictions = model.predict(features_valid)
    np_array = np.array(predictions)
    new_predictions = pd.Series(np_array, index = target_valid.index)
    print("RMSE =", mean_squared_error(target_valid,new_predictions) ** 0.5)
    print('Средний запас предсказанного сырья',new_predictions.mean())
    return new_predictions

In [27]:
new_predictions0 = rmse_prediction(features_train0, target_train0, features_valid0, target_valid0 )

RMSE = 37.61893312647744
Средний запас предсказанного сырья 92.67746991590629


In [28]:
new_predictions1 = rmse_prediction(features_train1, target_train1, features_valid1, target_valid1)

RMSE = 0.8915393978719793
Средний запас предсказанного сырья 68.93047860154645


In [29]:
new_predictions2 = rmse_prediction(features_train2, target_train2, features_valid2, target_valid2)

RMSE = 39.97593851112621
Средний запас предсказанного сырья 95.02403363016249


Вывод: самые точные предсказания мы получили при работе со вторым датасетом. В первом и третьем датасетах значение RMSE довольно высокое, что указывает на значительное расхождение между прогнозируемыми данными и реальными.

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [30]:
BUDGET = 10000000000
PRICE = 450000
OIL = 200
POINT = 500
BEST_POINT = 200

In [31]:
break_even = BUDGET/PRICE/OIL
break_even

111.11111111111111

Вывод: Стоит оценить запасы в скважинах, чтобы определить, насколько безубыточный объем существенно превышает основные объемы, содержащиеся в них.

In [32]:
print("25% скважин в первом регионе содержат сырья:",data0['product'].quantile(0.75))
print("16% скважин в втором регионе содержат сырья:",data1['product'].quantile(0.84))
print("25% скважин в третьем регионе содержат сырья:",data2['product'].quantile(0.75))

25% скважин в первом регионе содержат сырья: 128.56408897150277
16% скважин в втором регионе содержат сырья: 134.76630515769844
25% скважин в третьем регионе содержат сырья: 130.5950267155838


**выводы**

Мы поделили бюджет (10 млрд рублей) на доход от каждых 1000 баррелей (450 тыс. рублей = 450 * 1000 баррелей) и затем разделили на 200 скважин, которые мы выберем для разработки. Таким образом, мы получили средний объем запасов на каждую скважину, чтобы не уйти в убыток - 112 тысяч баррелей в среднем на скважину (округлено в большую сторону).

Если сравнить безубыточный объем сырья с средним запасом в каждом регионе, то можно заметить, что в каждом регионе средний объем запасов меньше безубыточного. Максимальный средний запас находится в третьем регионе и равен 95 тысячам баррелей на скважину, что не достаточно для выхода в прибыль.

Анализируя распределение объемов запасов по регионам на основе 3-го квантиля, мы замечаем, что в первом и третьем регионах 25% скважин достигают минимального порога безубыточности.

В то же время во втором регионе 3-й квантиль не достигает порога в 112 тысяч баррелей. Порог превышается только для 16% скважин.

In [33]:
def calculation(target, predictions, cost, count):
    probs_sorted = predictions.sort_values(ascending=False)
    deposits = target[probs_sorted.index][:count]
    return cost * deposits.sum() - BUDGET


## Расчёт прибыли и рисков

In [34]:
state = np.random.RandomState(12345)

In [35]:
values = []
for i in range(500):
    targetsome = target_valid0.sample(replace = True, random_state = state, n = POINT)
    predictionsome = new_predictions0[targetsome.index]
    values.append(calculation(targetsome,predictionsome,PRICE, BEST_POINT))

values = pd.Series(values)
print("Средняя выручка", values.mean())
print("Риск того, что будет убыточно", st.percentileofscore(values, 0))
print("95%-й Доверительный интервал: (", values.quantile(.025),",", values.quantile(.975),")")

Средняя выручка 424056697.5815736
Риск того, что будет убыточно 5.2
95%-й Доверительный интервал: ( -65714844.61684732 , 937944190.6030636 )


In [36]:
values = []
for i in range(500):
    targetsome = target_valid1.sample(replace = True, random_state = state, n = POINT)
    predictionsome = new_predictions1[targetsome.index]
    values.append(calculation(targetsome,predictionsome,PRICE, BEST_POINT))

values = pd.Series(values)
print("Средняя выручка", values.mean())
print("Риск того, что будет убыточно", st.percentileofscore(values, 0))
print("95%-й Доверительный интервал: (", values.quantile(.025),",", values.quantile(.975),")")

Средняя выручка 471307700.84493583
Риск того, что будет убыточно 2.2
95%-й Доверительный интервал: ( 23545096.96265221 , 872254045.3112618 )


In [37]:
values = []
for i in range(500):
    targetsome = target_valid2.sample(replace = True, random_state = state, n = POINT)
    predictionsome = new_predictions2[targetsome.index]
    values.append(calculation(targetsome,predictionsome,PRICE, BEST_POINT))

values = pd.Series(values)
print("Средняя выручка", values.mean())
print("Риск того, что будет убыточно", st.percentileofscore(values, 0))
print("95%-й Доверительный интервал: (", values.quantile(.025),",", values.quantile(.975),")")

Средняя выручка 395470417.8117764
Риск того, что будет убыточно 7.6000000000000005
95%-й Доверительный интервал: ( -166344418.20862022 , 936719611.1945325 )


In [38]:
for n, t in enumerate([target0, target1, target2], start=1):
    print(f'Среднее значение запасов для региона {n} = {np.round(t.mean(), 2)}')

Среднее значение запасов для региона 1 = 92.5
Среднее значение запасов для региона 2 = 68.83
Среднее значение запасов для региона 3 = 95.0


Вывод: Следует рассмотреть разработку второго региона, поскольку он представляет наименьший риск.

**Общий вывод**

Анализ данных показал отсутствие пропусков, дубликатов и других ненужных элементов. Так как столбец ID не использовался в работе, он был удален, и данные были подготовлены. Затем было необходимо масштабировать данные и обучить модель линейной регрессии. В результате были получены следующие результаты:

1 регион: RMSE = 37.61893312647744, Средний запас предсказанного сырья 92.67746991590556

2 регион: RMSE = 0.8915393978719723, Средний запас предсказанного сырья 68.93047860154668

3 регион: RMSE = 39.97593851112621, Средний запас предсказанного сырья 95.02403363016235

Самые точные прогнозы были получены на втором датасете. В то время как на первом и последнем датасетах показатель RMSE довольно высокий, что указывает на значительные расхождения между предсказанными и фактическими данными.

Затем мы произведем вычисления для определения средней выручки, средних объемов и риска, и получим следующие результаты:

1 регион: Средняя выручка 424056697.5815736, 95%-й Доверительный интервал (-65714844.61684732, 937944190.6030636), Риск убытков 5.2, Среднее значение объема скважин 92.5

2 регион: Средняя выручка 471307700.84493595, 95%-й Доверительный интервал (23545096.96265221, 872254045.3112618), Риск убытков 2.2, Среднее значение объема скважин 68.83

3 регион: Средняя выручка 395470417.81177634, 95%-й Доверительный интервал (-166344418.20862022, 936719611.1945325), Риск убытков 7.6, Среднее значение объема скважин 95.0

Рекомендуется рассмотреть разработку именно во **втором** регионе, поскольку он имеет наименьший риск.